In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


How can Keras API be used to create a custom CNN

### Convolution Operation
- Effect of Padding
- Kernel Size
- Strides

#### Unit Stride no padding
<figure>
<img src="http://deeplearning.net/software/theano/_images/numerical_no_padding_no_strides.gif">
</figure>

#### Zero padding unit strides
<img src ="http://deeplearning.net/software/theano/_images/arbitrary_padding_no_strides.gif">

### Realtionship between output size and padding, strides and kernel size
$n_{out}=\frac{n_{in}+2p-k}{s}+1$

Create a python function using this relationship

In [1]:
def conv_out(input,padding,kernel,strides):
    return (input+2*padding-kernel)/(strides)+1

Give an input of 5 pixel x 5 pixel, padding of 2px2p, kernel of 4px4p and a stride of 1

In [2]:
conv_out(input=5,padding=2,kernel=4,strides=1)

6.0

The output size will be 6p x 6p

Lets create another function that provides the kernel size given the other params

In [3]:
def get_k(output,input,strides,padding):
    return (strides*(1-output)+input+2*padding)

In [4]:
get_k(output=6,input=5,padding=2,strides=1)

4

Given the specifications, the size of the kernel should be 4px4p

In [5]:
import keras

<img src ='Lenet.PNG'>

Lets try to recreate this architecture using Keras API

In the first Conv layer we see input size is 32x32 and output is 28x28. In the Pooling layer we see input is 28x8 and output 14x14.

The question now is: What is the appropriate value of the kernel size so that the desired input-output relationships can be achieved?

In [6]:
## For Conv1 - assume single stride with no padding
get_k(input=32,output=28,strides=1,padding=0)

5

In [7]:
## For Pool1 - assume double stride with no padding
get_k(input=28,output=14,strides=2,padding=0)

2

In [8]:
## Conv2
get_k(input=14,output=10,strides=1,padding=0)

5

In [9]:
## Pool2
get_k(input=10,output=5,strides=2,padding=0)

2

Now we've gotten for each Conv and Pool layer the appropriate kernel size, we can now start assembling the model

In [10]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

In [11]:
## Create architecture
model=Sequential()
model.add(Conv2D(filters=6,kernel_size=(5,5),padding='valid',strides=(1,1),activation='relu',input_shape=(32,32,1)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=16,kernel_size=(5,5),padding='valid',strides=(1,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(120,activation='relu'))
model.add(Dense(84,activation='relu'))
model.add(Dense(10,activation='softmax'))


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

First Conv layer has an output size of 28x28 and first MaxPool layer has an output of 14x14 and so on as was desired by us